In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers


# 학습용 정상인 폐사진

In [2]:
train_x=[]
path='/Users/suminbae/Python-Workspace/kaggle/chest_xray/train/NORMAL/*.jpeg'

for filename in glob.glob(path):
    im=Image.open(filename).convert('L') # convert('L') - converts RGB images (if any) to grayscale
    img = im.resize((100, 100)) 
    train_x.append(tf.keras.preprocessing.image.img_to_array(img))
   

In [3]:
a=len(train_x)
train_y=np.zeros(a)

# 학습용 폐렴 사진

In [4]:
path='/Users/suminbae/Python-Workspace/kaggle/chest_xray/train/PNEUMONIA//*.jpeg'

for filename in glob.glob(path):
    im=Image.open(filename).convert('L')
    img=im.resize((100,100))
    train_x.append(tf.keras.preprocessing.image.img_to_array(img))
    
    

In [6]:
b=len(train_x)
train_y=np.concatenate((train_y,np.ones(b-a)))

In [7]:
train_x=np.stack(train_x)

# 평가용 데이터

In [8]:
val_x=[]
path='/Users/suminbae/Python-Workspace/kaggle/chest_xray/val/NORMAL/*.jpeg'

for filename in glob.glob(path):
    im=Image.open(filename).convert('L') # convert('L') - converts RGB images (if any) to grayscale
    img = im.resize((100, 100)) 
    val_x.append(tf.keras.preprocessing.image.img_to_array(img))

c=len(val_x)    
val_y=np.zeros(c)
    
path2='/Users/suminbae/Python-Workspace/kaggle/chest_xray/val/PNEUMONIA/*.jpeg'

for filename in glob.glob(path2):
    im=Image.open(filename).convert('L') # convert('L') - converts RGB images (if any) to grayscale
    img = im.resize((100, 100)) 
    val_x.append(tf.keras.preprocessing.image.img_to_array(img))
    
d=len(val_x)
val_y=np.concatenate((val_y,np.ones(d-c)))


In [9]:
val_x=np.stack(val_x)

# 테스트용 데이터 

In [10]:
test_x=[]
path='/Users/suminbae/Python-Workspace/kaggle/chest_xray/test/NORMAL/*.jpeg'

for filename in glob.glob(path):
    im=Image.open(filename).convert('L') # convert('L') - converts RGB images (if any) to grayscale
    img = im.resize((100, 100)) 
    test_x.append(tf.keras.preprocessing.image.img_to_array(img))
    
    
a=len(test_x)
test_y=np.zeros(a)


path2='/Users/suminbae/Python-Workspace/kaggle/chest_xray/test/PNEUMONIA/*.jpeg'

for filename in glob.glob(path2):
    im=Image.open(filename).convert('L') # convert('L') - converts RGB images (if any) to grayscale
    img = im.resize((100, 100)) 
    test_x.append(tf.keras.preprocessing.image.img_to_array(img))
    

b=len(test_x)
test_y=np.concatenate((test_y,np.ones(b-a)))
    



In [11]:
test_x=np.stack(test_x)

In [12]:
train_x.shape,val_x.shape,test_x.shape

((5216, 100, 100, 1), (16, 100, 100, 1), (624, 100, 100, 1))

In [13]:
tx,vx,ty,vy=train_test_split(train_x,train_y)

# model  

In [29]:
model=tf.keras.Sequential([
    layers.Conv2D(filters=60,kernel_size=(3,3),activation='relu',padding='same'),
    layers.MaxPooling2D(pool_size=(4,4)),
    layers.Dropout(0.3),
    
    layers.Conv2D(filters=40,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D(pool_size=(8,8)),
    layers.Dropout(0.3),
    
    layers.Flatten(),
    layers.Dense(512,activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
    
    
])

es=tf.keras.callbacks.EarlyStopping(monitor='loss',patience=5)


In [34]:
history=model.compile(loss="binary_crossentropy",optimizer='adam',metrics=['accuracy'])

In [35]:
model.fit(tx,ty,validation_data=(vx,vy),batch_size=300,epochs=15,callbacks=[es])

Train on 3912 samples, validate on 1304 samples
Epoch 1/15
3912/3912 [==============================] - 28s 7ms/sample - loss: 0.6125 - acc: 0.7326 - val_loss: 0.6839 - val_acc: 0.7446
Epoch 2/15
3912/3912 [==============================] - 28s 7ms/sample - loss: 0.5755 - acc: 0.7385 - val_loss: 0.6731 - val_acc: 0.7331
Epoch 3/15
3912/3912 [==============================] - 28s 7ms/sample - loss: 0.5467 - acc: 0.7485 - val_loss: 0.6385 - val_acc: 0.7538
Epoch 4/15
3912/3912 [==============================] - 28s 7ms/sample - loss: 0.5151 - acc: 0.7635 - val_loss: 0.5819 - val_acc: 0.8144
Epoch 5/15
3912/3912 [==============================] - 28s 7ms/sample - loss: 0.4444 - acc: 0.7922 - val_loss: 0.5051 - val_acc: 0.8459
Epoch 6/15
3912/3912 [==============================] - 28s 7ms/sample - loss: 0.3800 - acc: 0.8344 - val_loss: 0.4163 - val_acc: 0.8359
Epoch 7/15
3912/3912 [==============================] - 28s 7ms/sample - loss: 0.3529 - acc: 0.8428 - val_loss: 0.4382 - val_acc: 

# 대략 80퍼센트의 정확성을 가진다


In [38]:
model.evaluate(test_x,test_y)

624/624 [==============================] - 1s 2ms/sample - loss: 0.4232 - acc: 0.8125


[0.4232405152840492, 0.8125]

In [50]:
train_x[0].shape

(100, 100, 1)

In [60]:
cv2.imshow('pre',train_x[0])

#### https://github.com/siddarth-c/MachineLearning/blob/master/DeepLearning/PNEUMONIA%20or%20NOT.ipynb 